# kNN Hash Example

In [93]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter
from scipy.spatial import distance
from math import sqrt, ceil

## Iris dataset

In [94]:
df = load_iris()
df.data.shape

(150, 4)

In [95]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

* https://docs.python.org/2/library/functools.html Here you can read about "partial"
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html About mapping to [0,1]

In [96]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index
    
    def predict(self,u):
        min_d = 100
        for j in range(self.L):
            inds = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            
            for x in self.t_hh[j][0][sum(inds)]:
                temp = distance.euclidean(u,x[0])
                if(temp < min_d):
                    min_d = temp
            print(min_d) #nearest vector and its label
            #print(round(y/wt)) #label of u
            #Here you must put your code, extend the method with distance function and calculation with unknown sample "u"
            #Develop the rest part of kNN predict method that was discussed at the lecture

In [97]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target


In [98]:
knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict(test1x)
print("-------------")
print(test2y)
knnhash.predict(test2x)
print("-------------")
print(test3y)
knnhash.predict(test3x)

0
Counter({0: 22})
0.03254041656427031
Counter({0: 4, 1: 2})
0.03254041656427031
Counter({0: 15, 1: 8})
0.03254041656427031
Counter({0: 8, 1: 1})
0.03254041656427031
-------------
1
Counter({1: 6})
0.05007710104811091
Counter({2: 5, 1: 2})
0.05007710104811091
Counter({2: 3})
0.05007710104811091
Counter({2: 10, 1: 8})
0.05007710104811091
-------------
2
Counter({2: 10, 1: 6})
0.057940218203017
Counter({2: 2, 1: 1})
0.057940218203017
Counter({1: 7, 2: 4})
0.057940218203017
Counter({1: 4, 2: 4})
0.057940218203017


* Each string above corresponds to the particular hash table. And index in counter maps to the class. For example Counter({0: 13, 1: 1}) means that there are 13 samples close to "u" with "0" class labels and 1 sample with "1" class label.